IMPORT LIBRARIES

In [1]:
import cv2
import numpy as np
import  pandas as pd
import os
from pathlib import Path
import PIL as pil
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import time
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


# !sudo pip install opencv-python==3.4.2.17

# !sudo python3 pip install opencv-contrib-python==3.4.2.17

PRE-PROCESSING

In [2]:
def preprocessing(directory):
    images=[]
    current_dir = Path('Dataset/Drug Abuse/')
    entries= os.listdir(directory)
    for ent in entries:
        if ent != '.DS_Store':
            image=cv2.imread(os.getcwd()+"/"+directory+ent,0)
            #print(type(image))
            #image=image.tolist()
            resized_image = cv2.resize(image, (128,64), interpolation = cv2.INTER_AREA)
            #image_array=np.array(resized_image)
            images.append((resized_image))
    return images

In [3]:
images_d=preprocessing('Dataset/Drug Abuse/')
images_n=preprocessing('Dataset/Non Drug Abuse/')
images=images_d+images_n
print(len(images))

libpng warning: iCCP: profile 'icc': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'icc': 0h: PCS illuminant is not D50


196


In [4]:
def feature_extraction(images,method): 
    features=[]
    #print(type(features))
    for img in images: 
        if method=='HOG':
            fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=False)
            features.append(fd)
        if method=='SIFT':
            sift = cv2.xfeatures2d.SIFT_create()
            keypoints, descriptors = sift.detectAndCompute(img, None)
            if descriptors is not None:
                features.append(descriptors)  
                #features.append(descriptors)  
            #features = np.vstack(features)
    return features

In [5]:
images_features = feature_extraction(images,'SIFT')
images_features = np.vstack(images_features)
k = 150
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1)
flags = cv2.KMEANS_RANDOM_CENTERS
compactness, labels, centres = cv2.kmeans(images_features, k, None, criteria, 10, flags)


[ WARN:0] DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [6]:
def bag_of_features(features, centres, k = 500):
      vec = np.zeros((1, k))
      for i in range(features.shape[0]):
          feat = features[i]
          diff = np.tile(feat, (k, 1)) - centres
          dist = pow(((pow(diff, 2)).sum(axis = 1)), 0.5)
          idx_dist = dist.argsort()
          idx = idx_dist[0]
          vec[0][idx] += 1
      return vec

Bag Of Visual Words

In [7]:
def bovw(images):
    vec=[]
    labels=[]
    i=0
    #print(type(vect))
    for img in images:
        sift = cv2.xfeatures2d.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is not None:
            img_vec = bag_of_features(descriptors, centres, k)
            #print(type(vect))
            vec.append(img_vec)
            if i < len(images_d):
                labels.append(int(1))
            #features.append(descriptors)  
            else:
                labels.append(int(0))
        i=i+1
    vec= np.vstack(vec)
    return vec,labels

FINDING SIFT FEATURES

In [8]:
images_features_sift,labels=bovw(images)
print(len(labels))

196


FINDING HOG FEATURES

In [9]:
images_features_hog = feature_extraction(images,'HOG')
images_features_hog=np.vstack(images_features_hog)

SUPPORT VECTOR MACHINE

In [10]:

def SVM(dataf):
  X_train, X_test, y_train, y_test = train_test_split(dataf, labels, test_size=0.2)
  clf = SVC()
  clf.fit(X_train, y_train)
  preds = clf.predict(X_test)
  acc = accuracy_score(y_test, preds)
  conf_mat = confusion_matrix(y_test, preds)

  t1 = time.time()
  
  return acc*100, conf_mat

#acc,confusin_mat=SVM()


K Nearest Neighbour 

In [16]:

def KNN(dataf):
  X_train, X_test, y_train, y_test = train_test_split(dataf, labels, test_size=0.2)
  clf = KNeighborsClassifier(n_neighbors=7)
  clf.fit(X_train, y_train)
  preds = clf.predict(X_test)
  acc = accuracy_score(y_test, preds)
  conf_mat = confusion_matrix(y_test, preds)
  t1 = time.time()
  
  return acc*100, conf_mat


In [17]:
acc1_KNN,conf_mat1_KNN=KNN(images_features_sift)
print("KNN with SIFT Features: %d " %acc1_KNN)
acc2_KNN,conf_mat2_KNN=KNN(images_features_hog)
print("KNN with HOG Features: %d " %acc2_KNN)

KNN with SIFT Features: 67 
KNN with HOG Features: 60 


In [13]:
acc1_SVM,confusin_mat1_SVM=SVM(images_features_sift)
print("SVM with SIFT Features: %d " %acc1_SVM)

acc2_SVM,confusin_mat2_SVM=SVM(images_features_hog)
print("SVM with HOG Features: %d " %acc2_SVM)


SVM with SIFT Features: 87 
SVM with HOG Features: 80 
